In [1]:
!pip3 install lxml
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import random
import time
from lxml.html import fromstring


randos = ["https://sandiego.nextrequest.com/documents","https://sandiego.nextrequest.com/requests/new","https://sandiego.nextrequest.com/users/sign_in"]
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


PROXIES = list(get_proxies())

def get_data(url):
    
    def cleanhtml(raw_html):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext

    ids = url[-7:]
    
    rand_proxy = random.randrange(len(PROXIES))
    proxy = {'http':'http://'+PROXIES[rand_proxy],'https':'https://'+PROXIES[rand_proxy]}
    page = requests.get(url,headers = headers,proxies = proxy)
    soup = BeautifulSoup(page.content, 'html.parser')
    dept = soup.find_all(class_="current-department")
    depts = cleanhtml(str(dept[0])).strip()

    times = soup.find_all(class_="time-quotes")

    creation = cleanhtml(str(times[0])).strip()
    closing = cleanhtml(str(times[-1])).strip()
    
    
    def get_am_or_pm(time):
        if(time.partition("pm")[0][-2:] != 'am'):
#             print(time.partition("pm")[0][:-2],time.partition("pm")[0][:-2]!="am")
            return time.partition("pm")[0]+"pm"
        return time.partition("pm")[0]
    
    
    
    try:
        doj_creation = datetime.datetime.strptime(get_am_or_pm(creation), '%B %d, %Y, %I:%M%p')
        doj_closing  = datetime.datetime.strptime(get_am_or_pm(closing), '%B %d, %Y, %I:%M%p')
    except(ValueError):
        print(url)
        print(get_am_or_pm(creation))
        print(get_am_or_pm(closing))
    time_to_close = (doj_creation - doj_closing)

    return ids, depts, time_to_close


ids = []
depts = []
time_to_close = []

for i in range(21,22):
    for j in range(500,540):
#         time.sleep(15)
        url = "https://sandiego.nextrequest.com/requests/"+str(i) + "-" + str(j)
        rand_proxy = random.randrange(len(PROXIES))
        proxy = {'http':'http://'+ PROXIES[rand_proxy],'https':'https://'+PROXIES[rand_proxy]}
        if(j%19 ==0):
            rand_index = random.randrange(len(randos))
            requests.get(randos[rand_index])
        elif(requests.get(url,headers = headers,proxies = proxy).url != 'https://sandiego.nextrequest.com/requests'):
            print(url)
            req_id, dept, t = get_data(url)
            ids.append(req_id)
            depts.append(dept)
            time_to_close.append(t)

"""
TODO:
ids = []
depts = []
time_to_close = []
for url in urls:
    id, dept, t = get_data(url)
    ids.append(id)
    depts.append(dept)
    time_to_close.append(t)
result_df = pd.DataFrame(data = {'Request ID': ids,
                    'Department': depts,
                    'Time to Close': time_to_close}) 


"""

result_df = pd.DataFrame(data = {'Request ID': ids,
                    'Department': depts,
                    'Time to Close': time_to_close})


ValueError: check_hostname requires server_hostname

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import random
import time
from lxml.html import fromstring


randos = ["https://sandiego.nextrequest.com/documents","https://sandiego.nextrequest.com/requests/new","https://sandiego.nextrequest.com/users/sign_in"]
headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies


# PROXIES = list(get_proxies())

def get_data(url):
    
    def cleanhtml(raw_html):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext

    ids = url[-7:]
    
    # rand_proxy = random.randrange(len(PROXIES))
    # proxy = {'http':'http://'+PROXIES[rand_proxy],'https':'https://'+PROXIES[rand_proxy]}
    page = requests.get(url,headers = headers)#,proxies = proxy)
    soup = BeautifulSoup(page.content, 'html.parser')
    dept = soup.find_all(class_="current-department")
    depts = cleanhtml(str(dept[0])).strip()

    times = soup.find_all(class_="time-quotes")

    creation = cleanhtml(str(times[0])).strip()
    closing = cleanhtml(str(times[-1])).strip()
    
    
    def get_am_or_pm(time):
        if(time.partition("pm")[0][-2:] != 'am'):
#             print(time.partition("pm")[0][:-2],time.partition("pm")[0][:-2]!="am")
            return time.partition("pm")[0]+"pm"
        return time.partition("pm")[0]
    
    
    
    try:
        doj_creation = datetime.datetime.strptime(get_am_or_pm(creation), '%B %d, %Y, %I:%M%p')
        doj_closing  = datetime.datetime.strptime(get_am_or_pm(closing), '%B %d, %Y, %I:%M%p')
    except(ValueError):
        print(url)
        print(get_am_or_pm(creation))
        print(get_am_or_pm(closing))
    time_to_close = (doj_creation - doj_closing)

    return ids, depts, time_to_close


ids = []
depts = []
time_to_close = []

for i in range(21,22):
    for j in range(500,535):
#         time.sleep(15)
        url = "https://sandiego.nextrequest.com/requests/"+str(i) + "-" + str(j)
        # rand_proxy = random.randrange(len(PROXIES))
        # proxy = {'http':'http://'+ PROXIES[rand_proxy],'https':'https://'+PROXIES[rand_proxy]}
        if(j%19 ==0):
            rand_index = random.randrange(len(randos))
            requests.get(randos[rand_index])
        elif(requests.get(url,headers = headers).url != 'https://sandiego.nextrequest.com/requests'):
            print(url)
            req_id, dept, t = get_data(url)
            ids.append(req_id)
            depts.append(dept)
            time_to_close.append(t)

"""
TODO:
ids = []
depts = []
time_to_close = []
for url in urls:
    id, dept, t = get_data(url)
    ids.append(id)
    depts.append(dept)
    time_to_close.append(t)
result_df = pd.DataFrame(data = {'Request ID': ids,
                    'Department': depts,
                    'Time to Close': time_to_close}) 


"""

result_df = pd.DataFrame(data = {'Request ID': ids,
                    'Department': depts,
                    'Time to Close': time_to_close})


https://sandiego.nextrequest.com/requests/21-500
https://sandiego.nextrequest.com/requests/21-501
https://sandiego.nextrequest.com/requests/21-502
https://sandiego.nextrequest.com/requests/21-503
https://sandiego.nextrequest.com/requests/21-504
https://sandiego.nextrequest.com/requests/21-505
https://sandiego.nextrequest.com/requests/21-506
https://sandiego.nextrequest.com/requests/21-507
https://sandiego.nextrequest.com/requests/21-508
https://sandiego.nextrequest.com/requests/21-509
https://sandiego.nextrequest.com/requests/21-510
https://sandiego.nextrequest.com/requests/21-511
https://sandiego.nextrequest.com/requests/21-512
https://sandiego.nextrequest.com/requests/21-514
https://sandiego.nextrequest.com/requests/21-515
https://sandiego.nextrequest.com/requests/21-516
https://sandiego.nextrequest.com/requests/21-517
https://sandiego.nextrequest.com/requests/21-518
https://sandiego.nextrequest.com/requests/21-519
https://sandiego.nextrequest.com/requests/21-520
https://sandiego.nex

In [108]:
PROXIES = list(get_proxies())
rand_proxy = random.randrange(len(PROXIES))
proxy = {'http':'http://'+PROXIES[rand_proxy],'https':'https://'+PROXIES[rand_proxy]}

In [6]:
result_df

,Request ID,Department,Time to Close
0,/21-500,Department of Real Estate and Airport Management,37 days 07:25:00
1,/21-501,Animal Services,4 days 07:52:00
2,/21-502,City Council Administration,130 days 07:42:00
3,/21-503,Development Services,28 days 01:12:00
4,/21-504,Development Services,14 days 01:02:00
5,/21-505,Animal Services,5 days 23:20:00
6,/21-506,Public Utilities,32 days 22:23:00
7,/21-507,Police,11 days 15:57:00
8,/21-508,Department of Real Estate and Airport Management,25 days 13:14:00
9,/21-509,Engineering and Capital Projects,3 days 13:01:00


In [ ]:

import requests
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import random
import time
from lxml.html import fromstring
import numpy as np
from tqdm import tqdm

def get_data(url):
    
    def cleanhtml(raw_html):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext

    ids = url[-6:]
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    dept = soup.find_all(class_="current-department")
    depts = cleanhtml(str(dept[0])).strip()

    times = soup.find_all(class_="time-quotes")

    creation = cleanhtml(str(times[0])).strip()
    closing = cleanhtml(str(times[-1])).strip()
    
    def get_am_or_pm(time):
        if(time.partition("pm")[0][-2:] != 'am'):
            return time.partition("pm")[0]+"pm"
        return time.partition("pm")[0]
    
    
#     try:
    doj_creation = datetime.datetime.strptime(get_am_or_pm(creation), '%B %d, %Y, %I:%M%p')
    doj_closing  = datetime.datetime.strptime(get_am_or_pm(closing), '%B %d, %Y, %I:%M%p')
#     except(ValueError):
#         print(url)
#         print(get_am_or_pm(creation))
#         print(get_am_or_pm(closing))
    time_to_close = (doj_creation - doj_closing)

    return ids, depts, time_to_close


ids = []
depts = []
time_to_close = []

endings = ['21-' + str(x) for x in np.arange(10000)]
for i in tqdm(endings):
    url = "https://sandiego.nextrequest.com/requests/" + i 
    try:
        req_id, dept, t = get_data(url)
        ids.append(req_id)
        depts.append(dept)
        time_to_close.append(t)
    except:
        continue

result_df = pd.DataFrame(data = {'Request ID': ids,'Department': depts,'Time to Close': time_to_close})

result_df


 11%|█▏        | 1140/10000 [23:30<2:59:50,  1.22s/it]   

In [3]:
!pip freeze

alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client==1.7.2
anaconda-navigator==2.0.3
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1610472525955/work
anyio @ file:///opt/concourse/worker/volumes/live/64740ac7-3a9c-4fbb-6685-a51c4ff8b4ca/volume/anyio_1617783319350/work/dist
appdirs==1.4.4
applaunchservices==0.2.1
appnope @ file:///opt/concourse/worker/volumes/live/5f13e5b3-5355-4541-5fc3-f08850c73cf9/volume/appnope_1606859448618/work
appscript @ file:///opt/concourse/worker/volumes/live/82e8b4c7-2416-4d10-509e-144ca79d9b1d/volume/appscript_1611426996703/work
argh==0.26.2
argon2-cffi @ file:///opt/concourse/worker/volumes/live/d733ceb5-7f19-407b-7da7-a386540ab855/volume/argon2-cffi_1613037492998/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///opt/concourse/worker/volumes/live/343a8902-287c-47fb-6db8-923a63364302/volume/astroid_1613500849157/work
astropy @ file:///opt/concourse/worker/volumes/liv